In [9]:
import pandas as pd
from pydantic import Field, BaseModel
import json

In [3]:
resp_41_mini = pd.read_parquet("/Users/yangkc/working/llm/llm_domain_classification/data/intermediate/parsed_responses/openai/gpt-4.1-mini-2025-04-14.parquet")

In [6]:
resp_41_mini.query("rating < 0").sample(10)

,rating,domain,explanation,cost,input_token_count,output_token_count,cached_input_token_count,reasoning_token_count,provider,model
663,-1.0,sebgorka.com,I have no information or knowledge about the w...,0.000137,187,39,0,0,openai,gpt-4.1-mini-2025-04-14
2237,-1.0,rickwells.us,There is no available information or known rep...,0.000139,188,40,0,0,openai,gpt-4.1-mini-2025-04-14
1430,-1.0,usareally.com,There is no available information or credible ...,0.000134,186,37,0,0,openai,gpt-4.1-mini-2025-04-14
1624,-1.0,stwnewspress.com,There is no available information or known rep...,0.000159,189,52,0,0,openai,gpt-4.1-mini-2025-04-14
1524,-1.0,mediafiledc.com,I have no information or knowledge about the w...,0.000139,187,40,0,0,openai,gpt-4.1-mini-2025-04-14
527,-1.0,woay.com,I have no knowledge or information about the w...,0.000135,186,38,0,0,openai,gpt-4.1-mini-2025-04-14
117,-1.0,thejnotes.com,I have no information or knowledge about the w...,0.000137,187,39,0,0,openai,gpt-4.1-mini-2025-04-14
3785,-1.0,leaderpub.com,I have no information or knowledge about the w...,0.000134,186,37,0,0,openai,gpt-4.1-mini-2025-04-14
3624,-1.0,patribotics.blog,There is no available information or known cre...,0.000140,187,41,0,0,openai,gpt-4.1-mini-2025-04-14
378,-1.0,offthewire.com,I have no information about the credibility of...,0.000122,188,29,0,0,openai,gpt-4.1-mini-2025-04-14


In [8]:
class DomainRating(BaseModel):
    rating: float = Field(
        description="Domain credibility rating, ranging from -1 to 1",
        ge=-1,
        le=1,
    )
    domain: str = Field(description="The domain name.")
    explanation: str = Field(description="Explanation of the rating.")

In [10]:
def pydantic_model_to_openai_schema(pydantic_model, name) -> str:
    model_schema_dict = pydantic_model.model_json_schema()
    model_schema_dict['additionalProperties'] = False
    openai_schema = {
        "name": name,
        "strict": True,
        "schema": model_schema_dict
    }
    openai_schema_json_str = json.dumps(openai_schema, indent=2)
    return openai_schema_json_str

In [12]:
print(pydantic_model_to_openai_schema(DomainRating, 'domain_rating'))

{
  "name": "domain_rating",
  "strict": true,
  "schema": {
    "properties": {
      "rating": {
        "description": "Domain credibility rating, ranging from -1 to 1",
        "maximum": 1,
        "minimum": -1,
        "title": "Rating",
        "type": "number"
      },
      "domain": {
        "description": "The domain name.",
        "title": "Domain",
        "type": "string"
      },
      "explanation": {
        "description": "Explanation of the rating.",
        "title": "Explanation",
        "type": "string"
      }
    },
    "required": [
      "rating",
      "domain",
      "explanation"
    ],
    "title": "DomainRating",
    "type": "object",
    "additionalProperties": false
  }
}


In [ ]:
{
    "type": "json_schema",
    "name": "domain_rating",
    "strict": True,
    "schema": {
    "properties": {
        "rating": {
            "description": "Domain credibility rating, ranging from -1 to 1",
            "title": "Rating",
            "type": "number",
        },
        "domain": {
            "description": "The domain name",
            "title": "Domain",
            "type": "string",
        },
        "explanation": {
            "description": "Explanation of the rating.",
            "title": "Explanation",
            "type": "string",
        },
    },
    "additionalProperties": False,
    "required": ["rating", "domain", "explanation"],
    "title": "DomainRating",
    "type": "object",
}
}